In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("AlexanderBenady/lecture_summary_translations_english_spanish")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


PREPROCESS

In [ ]:
from transformers import MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = examples["Summary"]
    targets = examples["Summary_Spanish"]

    # Make sure that every element is a string
    inputs = [str(input) for input in inputs]
    targets = [str(target) for target in targets]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install transformers[torch]

In [ ]:
from sklearn.model_selection import train_test_split

# Let's assume tokenized_datasets["train"] is your full dataset
full_dataset = tokenized_datasets["train"]

# Split the dataset into training and evaluation datasets
# Ensure the datasets are formatted properly with continuous indices
from datasets import Dataset

# Convert dictionaries to Dataset objects
train_dataset = Dataset.from_dict(train_dataset)
eval_dataset = Dataset.from_dict(eval_dataset)

# Now you can shuffle and select to reset the indices if necessary
train_dataset = train_dataset.shuffle(seed=42).select(range(len(train_dataset)))
eval_dataset = eval_dataset.shuffle(seed=42).select(range(len(eval_dataset)))


restart runtime

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# TRAINING

In [ ]:
from transformers import Trainer, MarianMTModel

model = MarianMTModel.from_pretrained(model_name)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.093345
2,No log,0.089010
3,No log,0.087689


Epoch,Training Loss,Validation Loss
1,No log,0.085103
2,No log,0.083973
3,No log,0.083584


TrainOutput(global_step=150, training_loss=0.07762654622395833, metrics={'train_runtime': 74.1594, 'train_samples_per_second': 32.322, 'train_steps_per_second': 2.023, 'total_flos': 325017522929664.0, 'train_loss': 0.07762654622395833, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./Helsinki_EN_ES_withourdata")
tokenizer.save_pretrained("./Helsinki_EN_ES_withourdata")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


('./Helsinki_EN_ES_withourdata/tokenizer_config.json',
 './Helsinki_EN_ES_withourdata/special_tokens_map.json',
 './Helsinki_EN_ES_withourdata/vocab.json',
 './Helsinki_EN_ES_withourdata/source.spm',
 './Helsinki_EN_ES_withourdata/target.spm',
 './Helsinki_EN_ES_withourdata/added_tokens.json')

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model = MarianMTModel.from_pretrained("./Helsinki_EN_ES_withourdata")
tokenizer = MarianTokenizer.from_pretrained("./Helsinki_EN_ES_withourdata")

# Create text for translation
text_to_translate = [
    "Good morning, everyone.",
    "Today, we're delving into the fascinating world of Apache Kafka.",
    "This lecture will provide you with a comprehensive understanding of what Kafka is, its architecture, key concepts, and its significance in modern data processing systems.",
    "So, let's start with the basics. What exactly is Apache Kafka?",
    "Kafka is an open-source distributed event streaming platform. It is designed to handle real-time data feeds with high-throughput, fault tolerance, and scalability in mind.",
    "Originally developed by LinkedIn, it has now become an integral part of many data-centric organizations.",
    "Now, let's dive into the architecture of Kafka. At its core, Kafka has four main components: producers, topics, brokers, and consumers.",
    "Producers are responsible for publishing messages to Kafka topics.",
    "Topics are the channels through which data is organized and stored.",
    "Brokers are the Kafka servers responsible for handling message storage and distribution.",
    "Consumers subscribe to topics and process the data.",
    "Moving on to some key concepts of Kafka. One crucial concept is the 'partition'. Topics in Kafka are divided into partitions, allowing for parallel processing and scalability. Each partition is replicated across multiple brokers for fault tolerance.",
    "Another important concept is 'offsets'. Every message within a partition is assigned a unique offset, which serves as a kind of message identifier. Consumers can keep track of their progress by storing the offsets they have processed.",
    "Now, let's talk about some common use cases for Apache Kafka. Kafka is widely used for real-time stream processing, log aggregation, event sourcing, and data integration across various systems.",
    "Its ability to handle high-throughput data streams makes it invaluable in scenarios such as financial trading, social media analytics, and Internet of Things (IoT) applications.",
    "In conclusion, Apache Kafka is a powerful distributed streaming platform that has revolutionized the way we handle real-time data. Its scalable architecture, fault tolerance, and versatility make it a popular choice for organizations dealing with large volumes of data.",
    "By understanding Kafka's core concepts and architecture, you'll be well-equipped to leverage its capabilities in your own projects.",
    "Thank you for your attention, and I'll now open the floor for any questions you may have."
]
translations = []

for sentence in text_to_translate:
    # Tokenize the text
    tokenized_text = tokenizer(sentence, return_tensors="pt", padding=True)

    # Generate translation
    translated_tokens = model.generate(**tokenized_text)

    # Decode the translated text
    translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    translations.append(translation)

# Now you have a list of translations
for original, translated in zip(text_to_translate, translations):
    print(f'Original: {original}')
    print(f'Translated: {translated}')
    print('')  # Print a new line for better readability


Original: Good morning, everyone.
Translated: Buenos días a todos.

Original: Today, we're delving into the fascinating world of Apache Kafka.
Translated: Hoy, estamos profundizando en el fascinante mundo de Apache Kafka.

Original: This lecture will provide you with a comprehensive understanding of what Kafka is, its architecture, key concepts, and its significance in modern data processing systems.
Translated: Esta conferencia le proporcionará una comprensión completa de lo que es Kafka, su arquitectura, conceptos clave y su importancia en los modernos sistemas de procesamiento de datos.

Original: So, let's start with the basics. What exactly is Apache Kafka?
Translated: Así que, empecemos con lo básico. ¿Qué es exactamente Apache Kafka?

Original: Kafka is an open-source distributed event streaming platform. It is designed to handle real-time data feeds with high-throughput, fault tolerance, and scalability in mind.
Translated: Kafka es una plataforma de streaming de eventos distri

# TEST WITH ORIGINAL HELSINKI MODEL

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
src_text = [
    "Good morning, everyone.",
    "Today, we're delving into the fascinating world of Apache Kafka.",
    "This lecture will provide you with a comprehensive understanding of what Kafka is, its architecture, key concepts, and its significance in modern data processing systems.",
    "So, let's start with the basics. What exactly is Apache Kafka?",
    "Kafka is an open-source distributed event streaming platform. It is designed to handle real-time data feeds with high-throughput, fault tolerance, and scalability in mind.",
    "Originally developed by LinkedIn, it has now become an integral part of many data-centric organizations.",
    "Now, let's dive into the architecture of Kafka. At its core, Kafka has four main components: producers, topics, brokers, and consumers.",
    "Producers are responsible for publishing messages to Kafka topics.",
    "Topics are the channels through which data is organized and stored.",
    "Brokers are the Kafka servers responsible for handling message storage and distribution.",
    "Consumers subscribe to topics and process the data.",
    "Moving on to some key concepts of Kafka. One crucial concept is the 'partition'. Topics in Kafka are divided into partitions, allowing for parallel processing and scalability. Each partition is replicated across multiple brokers for fault tolerance.",
    "Another important concept is 'offsets'. Every message within a partition is assigned a unique offset, which serves as a kind of message identifier. Consumers can keep track of their progress by storing the offsets they have processed.",
    "Now, let's talk about some common use cases for Apache Kafka. Kafka is widely used for real-time stream processing, log aggregation, event sourcing, and data integration across various systems.",
    "Its ability to handle high-throughput data streams makes it invaluable in scenarios such as financial trading, social media analytics, and Internet of Things (IoT) applications.",
    "In conclusion, Apache Kafka is a powerful distributed streaming platform that has revolutionized the way we handle real-time data. Its scalable architecture, fault tolerance, and versatility make it a popular choice for organizations dealing with large volumes of data.",
    "By understanding Kafka's core concepts and architecture, you'll be well-equipped to leverage its capabilities in your own projects.",
    "Thank you for your attention, and I'll now open the floor for any questions you may have."
]

model_name = "Helsinki-NLP/opus-mt-tc-big-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

for t in translated:
    print( tokenizer.decode(t, skip_special_tokens=True) )

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/804k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/824k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/466M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Buenos días a todos.
Hoy nos adentramos en el fascinante mundo de Apache Kafka.
Esta conferencia le proporcionará una comprensión completa de lo que es Kafka, su arquitectura, conceptos clave y su importancia en los sistemas modernos de procesamiento de datos.
Empecemos con lo básico. ¿Qué es exactamente Apache Kafka?
Kafka es una plataforma de transmisión de eventos distribuidos de código abierto. Está diseñada para manejar fuentes de datos en tiempo real con alto rendimiento, tolerancia a fallas y escalabilidad en mente.
Originalmente desarrollado por LinkedIn, ahora se ha convertido en una parte integral de muchas organizaciones centradas en los datos.
Ahora, vamos a sumergirnos en la arquitectura de Kafka. En su núcleo, Kafka tiene cuatro componentes principales: productores, temas, corredores y consumidores.
Los productores son responsables de publicar mensajes a los temas de Kafka.
Los temas son los canales a través de los cuales se organizan y almacenan los datos.
Los corredores

# TEST WITH OUR FINE-TUNED MODEL

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
src_text = [
    "Good morning, everyone.",
    "Today, we're delving into the fascinating world of Apache Kafka.",
    "This lecture will provide you with a comprehensive understanding of what Kafka is, its architecture, key concepts, and its significance in modern data processing systems.",
    "So, let's start with the basics. What exactly is Apache Kafka?",
    "Kafka is an open-source distributed event streaming platform. It is designed to handle real-time data feeds with high-throughput, fault tolerance, and scalability in mind.",
    "Originally developed by LinkedIn, it has now become an integral part of many data-centric organizations.",
    "Now, let's dive into the architecture of Kafka. At its core, Kafka has four main components: producers, topics, brokers, and consumers.",
    "Producers are responsible for publishing messages to Kafka topics.",
    "Topics are the channels through which data is organized and stored.",
    "Brokers are the Kafka servers responsible for handling message storage and distribution.",
    "Consumers subscribe to topics and process the data.",
    "Moving on to some key concepts of Kafka. One crucial concept is the 'partition'. Topics in Kafka are divided into partitions, allowing for parallel processing and scalability. Each partition is replicated across multiple brokers for fault tolerance.",
    "Another important concept is 'offsets'. Every message within a partition is assigned a unique offset, which serves as a kind of message identifier. Consumers can keep track of their progress by storing the offsets they have processed.",
    "Now, let's talk about some common use cases for Apache Kafka. Kafka is widely used for real-time stream processing, log aggregation, event sourcing, and data integration across various systems.",
    "Its ability to handle high-throughput data streams makes it invaluable in scenarios such as financial trading, social media analytics, and Internet of Things (IoT) applications.",
    "In conclusion, Apache Kafka is a powerful distributed streaming platform that has revolutionized the way we handle real-time data. Its scalable architecture, fault tolerance, and versatility make it a popular choice for organizations dealing with large volumes of data.",
    "By understanding Kafka's core concepts and architecture, you'll be well-equipped to leverage its capabilities in your own projects.",
    "Thank you for your attention, and I'll now open the floor for any questions you may have."
]

model = MarianMTModel.from_pretrained("./Helsinki_EN_ES_withourdata")
tokenizer = MarianTokenizer.from_pretrained("./Helsinki_EN_ES_withourdata")
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

for t in translated:
    print( tokenizer.decode(t, skip_special_tokens=True) )

Buenos días a todos.
Hoy, estamos profundizando en el fascinante mundo de Apache Kafka.
Esta conferencia le proporcionará una comprensión completa de lo que es Kafka, su arquitectura, conceptos clave y su importancia en los modernos sistemas de procesamiento de datos.
Así que, empecemos con lo básico. ¿Qué es exactamente Apache Kafka?
Kafka es una plataforma de streaming de eventos distribuida de código abierto. Está diseñada para manejar fuentes de datos en tiempo real con alto rendimiento, tolerancia a fallos y escalabilidad en mente.
Originalmente desarrollado por LinkedIn, se ha convertido en una parte integral de muchas organizaciones centradas en los datos.
Ahora, sumerjamos en la arquitectura de Kafka. En su núcleo, Kafka tiene cuatro componentes principales: productores, temas, corredores y consumidores.
Los productores son responsables de la publicación de mensajes a los temas de Kafka.
Los temas son los canales a través de los cuales se organizan y almacenan los datos.
Los co

In [ ]:
 !tar -czvf Helsinki_EN_ES_withourdata.tar.gz Helsinki_EN_ES_withourdata/


Helsinki_EN_ES_withourdata/
Helsinki_EN_ES_withourdata/generation_config.json
Helsinki_EN_ES_withourdata/source.spm
Helsinki_EN_ES_withourdata/model.safetensors
Helsinki_EN_ES_withourdata/target.spm
Helsinki_EN_ES_withourdata/config.json
Helsinki_EN_ES_withourdata/.ipynb_checkpoints/
Helsinki_EN_ES_withourdata/special_tokens_map.json
Helsinki_EN_ES_withourdata/tokenizer_config.json
Helsinki_EN_ES_withourdata/vocab.json


In [ ]:
from google.colab import files
files.download('Helsinki_EN_ES_withourdata.tar.gz')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# IMPLEMENTATION

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "sfarjebespalaia/enestranslatorforsummaries"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

src_text = [
    "By understanding Kafka's core concepts and architecture, you'll be well-equipped to leverage its capabilities in your projects.",
    "Thank you for your attention, and I'll now open the floor for any questions you may have."
]

# Prepare the text data into the format that the model expects
tokenized_text = tokenizer(src_text, return_tensors="pt", padding=True)

# Generate the translation using the model
translated = model.generate(**tokenized_text)

# Decode the translated text
for t in translated:
    print(tokenizer.decode(t, skip_special_tokens=True))